In [196]:
import pandas as pd
from binance.client import Client
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, precision_score, recall_score
import pickle

In [197]:
key = 'GSwEjjrOWm4m9vU81CSm25kxOvMe5eGKZXIpKfOlPKYCIkAgkbpBLSuXitUt7Hgm'
secret = 'GdrlCQK1leZNQ60njgGh8sfLwsy1ppngYfPI33CzFlNKCTCO8HyjM3kEFneR6s1O'

In [198]:
currency = 'ETHUSDT'
client = Client(key, secret)
bars = client.get_historical_klines(currency, Client.KLINE_INTERVAL_1HOUR, "1 year ago MSK")
for line in bars:
    del line[5:]
df = pd.DataFrame(bars, columns=['date', 'open', 'high', 'low', 'close'])

In [199]:
df.close = df.close.astype('float32')
df.open = df.open.astype('float32')
df

,date,open,high,low,close
0,1591210800000,241.759995,243.00000000,239.46000000,240.800003
1,1591214400000,240.850006,241.99000000,240.11000000,241.699997
2,1591218000000,241.699997,242.23000000,240.80000000,241.419998
3,1591221600000,241.380005,243.41000000,241.18000000,243.119995
4,1591225200000,243.149994,245.00000000,242.42000000,244.520004
...,...,...,...,...,...
8740,1622728800000,2778.389893,2803.00000000,2765.47000000,2792.979980
8741,1622732400000,2792.979980,2818.35000000,2774.57000000,2788.110107
8742,1622736000000,2788.110107,2839.00000000,2778.72000000,2831.399902
8743,1622739600000,2831.689941,2850.00000000,2819.08000000,2822.689941


In [200]:
df['y'] = df.close - df.open
df["y'"] = [0 if i[4] < i[1] else 1 for i in df.values]
df

,date,open,high,low,close,y,y'
0,1591210800000,241.759995,243.00000000,239.46000000,240.800003,-0.959991,0
1,1591214400000,240.850006,241.99000000,240.11000000,241.699997,0.849991,1
2,1591218000000,241.699997,242.23000000,240.80000000,241.419998,-0.279999,0
3,1591221600000,241.380005,243.41000000,241.18000000,243.119995,1.739990,1
4,1591225200000,243.149994,245.00000000,242.42000000,244.520004,1.370010,1
...,...,...,...,...,...,...,...
8740,1622728800000,2778.389893,2803.00000000,2765.47000000,2792.979980,14.590088,1
8741,1622732400000,2792.979980,2818.35000000,2774.57000000,2788.110107,-4.869873,0
8742,1622736000000,2788.110107,2839.00000000,2778.72000000,2831.399902,43.289795,1
8743,1622739600000,2831.689941,2850.00000000,2819.08000000,2822.689941,-9.000000,0


In [201]:
Y = df.y
Y_sign = df["y'"]

In [202]:
PAST = 28
FUTURE = 1
START = PAST
END = len(df.date) - FUTURE

In [203]:
df.low = df.low.astype('float32')
df.high = df.high.astype('float32')

In [204]:
open_, high, low, close = df.open, df.high, df.low, df.close

In [205]:
new_df = []
for i in range(START, END+1):
    new_df.append(list(open_[i-PAST:i]) + list(high[i-PAST:i]) + list(low[i-PAST:i]) + list(close[i-PAST:i]))

In [206]:
pc = []
for i in range(PAST):
    pc.append(f"past_open_{i}")
for i in range(PAST):
    pc.append(f"past_high_{i}")
for i in range(PAST):
    pc.append(f"past_low_{i}")
for i in range(PAST):
    pc.append(f"past_close_{i}")

In [207]:
tdf = pd.DataFrame(new_df, columns = (pc))

In [208]:
tdf

,past_open_0,past_open_1,past_open_2,past_open_3,past_open_4,past_open_5,past_open_6,past_open_7,past_open_8,past_open_9,...,past_close_18,past_close_19,past_close_20,past_close_21,past_close_22,past_close_23,past_close_24,past_close_25,past_close_26,past_close_27
0,241.759995,240.850006,241.699997,241.380005,243.149994,244.539993,243.820007,244.619995,246.000000,245.339996,...,245.000000,243.380005,245.000000,243.070007,243.990005,240.389999,241.470001,241.759995,242.110001,243.339996
1,240.850006,241.699997,241.380005,243.149994,244.539993,243.820007,244.619995,246.000000,245.339996,244.490005,...,243.380005,245.000000,243.070007,243.990005,240.389999,241.470001,241.759995,242.110001,243.339996,243.210007
2,241.699997,241.380005,243.149994,244.539993,243.820007,244.619995,246.000000,245.339996,244.490005,243.369995,...,245.000000,243.070007,243.990005,240.389999,241.470001,241.759995,242.110001,243.339996,243.210007,243.559998
3,241.380005,243.149994,244.539993,243.820007,244.619995,246.000000,245.339996,244.490005,243.369995,244.059998,...,243.070007,243.990005,240.389999,241.470001,241.759995,242.110001,243.339996,243.210007,243.559998,244.399994
4,243.149994,244.539993,243.820007,244.619995,246.000000,245.339996,244.490005,243.369995,244.059998,243.369995,...,243.990005,240.389999,241.470001,241.759995,242.110001,243.339996,243.210007,243.559998,244.399994,243.580002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8712,2684.510010,2681.159912,2694.389893,2701.100098,2732.580078,2767.280029,2791.070068,2760.000000,2769.540039,2772.379883,...,2767.169922,2816.750000,2846.689941,2820.320068,2836.000000,2874.469971,2874.750000,2806.629883,2826.560059,2778.389893
8713,2681.159912,2694.389893,2701.100098,2732.580078,2767.280029,2791.070068,2760.000000,2769.540039,2772.379883,2758.189941,...,2816.750000,2846.689941,2820.320068,2836.000000,2874.469971,2874.750000,2806.629883,2826.560059,2778.389893,2792.979980
8714,2694.389893,2701.100098,2732.580078,2767.280029,2791.070068,2760.000000,2769.540039,2772.379883,2758.189941,2722.209961,...,2846.689941,2820.320068,2836.000000,2874.469971,2874.750000,2806.629883,2826.560059,2778.389893,2792.979980,2788.110107
8715,2701.100098,2732.580078,2767.280029,2791.070068,2760.000000,2769.540039,2772.379883,2758.189941,2722.209961,2718.330078,...,2820.320068,2836.000000,2874.469971,2874.750000,2806.629883,2826.560059,2778.389893,2792.979980,2788.110107,2831.399902


In [209]:
y = Y[START:END+1]

In [194]:
X_train = tdf[pc][:-100]
y_train = y[:-100]
X_test = tdf[pc][-100:]
y_test = y[-100:]
acc = []
for i in range(4, 5):
    print(i)
    model = RandomForestRegressor(random_state = 42, n_estimators = i*10, n_jobs=8).fit(X_train, y_train)
    print(i)
    pred = model.predict(X_test)
    acc.append([mean_absolute_error(pred, y_test), i])
    print(i)
acc = sorted(acc, key = lambda x: x[0])
print(acc[0], acc[-1])

4


ValueError: Found input variables with inconsistent numbers of samples: [8618, 8617]

In [153]:
model = RandomForestRegressor(random_state = 42, n_estimators = 40, n_jobs=8).fit(tdf[pc], y)
filename = f'models/{currency}_value.sav'
pickle.dump(model, open(filename, 'wb'))

-0.12262625688781781

In [215]:
y_s = Y_sign[START:END+1]
X_train = tdf[pc][:-1717]
y_train_s = y_s[:-1717]
X_test = tdf[pc][-1717:-1617]
y_test_s = y_s[-1717:-1617]
acc = []
for i in range(10, 11):
    model = RandomForestClassifier(random_state = 42, n_estimators = i*10, n_jobs=8).fit(X_train, y_train_s)
    pred = model.predict_proba(X_test)
    pred = [1 if j[1] > 0.45 else 0 for j in pred]
    acc.append([precision_score(y_test_s, pred), recall_score(y_test_s, pred), i])
acc = sorted(acc, key = lambda x: x[0])
print(acc[0], acc[-1])

[0.45454545454545453, 0.39215686274509803, 10] [0.45454545454545453, 0.39215686274509803, 10]


In [150]:
#model =RandomForestClassifier(random_state = 42, n_estimators = 100, n_jobs=8).fit(tdf[pc], y_s)
filename = f'models/{currency}_rise.sav'
#pickle.dump(model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test_s)

In [219]:
client = Client(key, secret)
client.API_URL = 'https://testnet.binance.vision/api'
balance = client.get_account(recvWindow=60000)['balances']

In [221]:
for i in balance:
    print(i['asset'], i['free'])

BNB 1000.00000000
BTC 1.00000000
BUSD 10000.00000000
ETH 101.81422000
LTC 500.00000000
TRX 500000.00000000
USDT 4682.96725166
XRP 50000.00000000
